In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.getcwd()
PATH = os.getcwd() + '/drive/MyDrive/Colab Notebooks/AI_Hackathon'
#PATH = os.getcwd() + '/drive/MyDrive/AI_Hackathon' # seeun
os.chdir(PATH)
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/AI_Hackathon'

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# 토크나이저 관련 경고 무시
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

사용 디바이스: cuda


In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

voice_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_Hackathon/data/voice_df_f.csv', sep='\t')

In [ ]:
voice_df.isnull().sum()

Unnamed: 0    0
content       0
target        0
dtype: int64

In [ ]:
from sklearn.utils import resample
import pandas as pd

# voice_df 데이터프레임에서 content와 target 컬럼 추출
X = voice_df['content']
y = voice_df['target']

# 클래스 0와 클래스 1을 분리
class_0 = voice_df[voice_df['target'] == 0]
class_1 = voice_df[voice_df['target'] == 1]

# 클래스 1을 2배 복제하여 오버샘플링
oversampled_class_1 = resample(class_1, replace=True, n_samples=len(class_1) * 2)

# 클래스 0와 클래스 1을 합쳐 오버샘플링된 데이터 생성
voice = pd.concat([class_0, oversampled_class_1])


In [ ]:
class_0.sum()

Unnamed: 0                                              3107829
content       친애하다 판사 라는 관련 드라마무한도전 완전 폐지 돼다 없어지다 무한도전 자주 자다...
target                                                        0
dtype: object

In [ ]:
class_0

,Unnamed: 0,content,target
1012,1012,친애하다 판사 라는 관련 드라마,0
1013,1013,무한도전 완전 폐지 돼다 없어지다 무한도전 자주 자다 챙기다,0
1014,1014,따다,0
1015,1015,지다 호랑 계단 올라오다 운동 대요 서다 운동,0
1016,1016,뭔가 효과,0
...,...,...,...
2686,2686,개월 개월 할부 해주다 카드 번호 고요 비밀번호 자리 땡땡 이제 가입 결제,0
2687,2687,년생 년생 이니까 년생 년생 할인 넣다 가입 쿠폰 해주다 강의 신청 가능하다 신청 ...,0
2688,2688,아이디 다빈 해주다 비밀번호 케이 지다 김표 느낌표 해주다 이메일 자주 쓰다,0
2689,2689,문자 보내다 인강 듣다 방법 시간표 보내다 교재 최대한 빨리 보내다 수업 주일 주일...,0


In [ ]:
class_1.sum()

Unnamed: 0                                               511566
content       은행권 들어가다 나오다 태도 갑자기 이렇다 본인 관할 경찰서 기범 여보세요 피해자 ...
target                                                     1012
dtype: object

In [ ]:
voice.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df_shuffled = sklearn.utils.shuffle(voice)

train, test = train_test_split(voice, test_size=0.2)

In [ ]:
train

,content,target
107,성립 본인 께서 선택 사항 근무 집행 방 성립 사용 한번 협상 진행 녹취 부분 계속...,1
82,기범 직급 과장 이세 피해자 기범 과장 께서 본인 명의 돼다 서류 서도 처분 겁니다...,1
465,소비 대다 계약 공증 발건 진행 드릴 피해자 기범 신청 해드리다 공증 비용 쩍게 프...,1
651,범용 피해자 내다 기범 피해자 갖다 그게 돼다 기범 대리 접수 고객 드리다 피해자 ...,1
768,어떻다 걱정 말고 빨리 기기 괜히 빨리 본사 동해 본인 빨리 끝내다 어떻다 확인 가...,1
...,...,...
992,기범 마지막 법률 본인 한테 고지 해드리다 본인 께서 라도 사건 다거나 은닉 형법 ...,1
2183,맞다 그리다 신청 명단 제출 나다 교재 수령,0
6,기범 이동 어떻다 교통 수단 떤다 이용 하나요 피해자 걸다 갈다 농협 기범 걸리다 ...,1
808,제출 그게 건가 대다 출금 오늘 고객 께서 한도 떨다 오늘 출금 까지는 늦다 까지는...,1


In [ ]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TokenDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['content']
        label = self.data.iloc[idx]['target']

        # 토큰화 처리
        tokens = self.tokenizer(
            text,
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


In [ ]:
# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)

In [ ]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [ ]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [ ]:
# CustomBertModel 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)

            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
import numpy as np

# 최대 Epoch을 지정합니다.
num_epochs = 3

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 0.09419, training accuracy: 0.99561: 100%|██████████| 371/371 [04:31<00:00,  1.36batch/s]


[INFO] val_loss has been improved from inf to 0.00007. Saving Model!
epoch 01, loss: 0.01180, acc: 0.99561, val_loss: 0.00007, val_accuracy: 1.00000


training loss: 0.00145, training accuracy: 1.00000: 100%|██████████| 371/371 [04:37<00:00,  1.34batch/s]


[INFO] val_loss has been improved from 0.00007 to 0.00002. Saving Model!
epoch 02, loss: 0.00018, acc: 1.00000, val_loss: 0.00002, val_accuracy: 1.00000


training loss: 0.00068, training accuracy: 1.00000: 100%|██████████| 371/371 [04:36<00:00,  1.34batch/s]


[INFO] val_loss has been improved from 0.00002 to 0.00001. Saving Model!
epoch 03, loss: 0.00008, acc: 1.00000, val_loss: 0.00001, val_accuracy: 1.00000


In [ ]:
# 저장한 state_dict를 로드 합니다.
bert.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

In [ ]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [ ]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '일반 대화 입니다.',
    1: '보이스피싱 입니다.'
}

# CustomPredictor 인스턴스를 생성
predictor = CustomPredictor(bert, tokenizer, labels)

In [ ]:
import pandas as pd
import re
#okt 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

#pykoSpacing 띄어쓰기
# from pykoSpacing import Spacing
# spacing = Spacing()

# from hanspell import spell_checker


#stopwords list txt file
# with open('/content/drive/MyDrive/Colab Notebooks/AI_Hackathon/stopwords.txt', 'r') as f:
#   list_file = f.readlines()
with open('stopwords.txt', 'r') as f:
  list_file = f.readlines()  # seeun
stopwords = list_file[0].split(" ")


#각자 노트북마다 DATA_PATH가 조금씩 달라져 안되는 경우가 있습니다.(colab 공유 후 문제같음)
#밑에 두개 다 시도해보시고 안되면 경로 확인 부탁드립니다
#DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/AI_Hackathon/data'
DATA_PATH = 'data'


#현재 여러가지 시도하느라 함수들이 중구난방입니다 -> 전처리 함수 나중에 따로 만들어서 합칠예정

#txt file 읽어오기용-encoding=UTF8
def read_txt_file(directory):
  scripts = {}
  scripts["content"] = []

  for file in os.listdir(directory):
    if 'txt' in file:
      script = open(os.path.join(directory, file), 'r', encoding='UTF8')
      content = script.readlines()
      scripts["content"].append(content)

  return pd.DataFrame.from_dict(scripts)

#txt file 읽어오기용-encoding=cp949
def read_txt_file2(directory):
  scripts = {}
  scripts["content"] = []

  for file in os.listdir(directory):
    if 'txt' in file:
      script = open(os.path.join(directory, file), 'r', encoding='cp949')
      content = script.readlines()
      scripts["content"].append(content)

  return pd.DataFrame.from_dict(scripts)

#cleansing - 한글만 추출, 소문자화, 맞춤법(hanspell)
def cleansing(content):
  # 기호 제거하고 한글만 추출
  # hangul = re.compile('[가-힣]+').findall(content)
  # text = ' '.join(hangul)
  t = re.sub("[^가-힣]+", " ", content)

  #lower case
  result = t.lower()
  #맞춤법까지 추가하면 cleansing 완료

  return result

#text형태 : list
def extract_words(text):
  words = okt.normalize(text)
  morph_words = okt.morphs(words, stem=True)
  stopwords = list_file[0].split(" ")
  remove_stopwords = [x for x in morph_words if x not in stopwords]

  one_word = [x for x in remove_stopwords if len(x) > 1 or x=='방' or x=='신' or x=='톡']
  result = " ".join(one_word)
  return result

#이게 최종 전처리 함수인데 아직 미완성 추후 정리예정
#기능(읽어왔다는 전제)-cleansing진행->형태소분석->불용어제거

def preprocessing(text):
  result = cleansing(text)
  result = result.replace(" ","") #효과적인 띄어쓰기를 위해 다 붙임
  #띄어쓰기
  # result = spacing(result)
  words = extract_words(result) #간단한 맞춤법 교정


  return words

In [ ]:
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    pre_sentence = preprocessing(input_sentence)
    predictor.predict(pre_sentence)

In [ ]:
# 예측할 대화 입력
predict_sentence(predictor)

문장을 입력해 주세요: 안녕, 오랜만이야. | 어떻게 지내? | 안녕! | 나도 오랜만이야. | 잘 지내고 있어. | 너는 어때? | 나도 괜찮아. | 일상생활이 바빠서 좀 피곤해. | 이해해. | 업무나 학교 어때? | 무슨 일 하고 있어?
[일반 대화 입니다.]
확률은: 98.678% 입니다.
